In [11]:
# Part 1 Setup & Imports #
!pip install biopython
from Bio import SeqIO
import pandas as pd
import sqlite3

In [12]:
# Part 2 Preparation -- define a new function #
def execute_sql_command(command: str,database_name: str = "lab3.db") -> None:
    conn = sqlite3.connect(database_name)
    c = conn.cursor()
    c.execute(command)
    conn.commit()
# Part 2.1 Create a table for pathways #
execute_sql_command("""
CREATE TABLE IF NOT EXISTS pathways (
    pathway_id INTEGER PRIMARY KEY AUTOINCREMENT,
    pathway_name TEXT NOT NULL,
    pathway_description TEXT
);
""")
# Part 2.2 Creat a table for enzymes #
execute_sql_command("DROP TABLE IF EXISTS enzymes;")
execute_sql_command("""
CREATE TABLE IF NOT EXISTS enzymes (
    enzyme_id INTEGER PRIMARY KEY AUTOINCREMENT,
    enzyme_commission TEXT NOT NULL,
    enzyme_name TEXT,
    enzyme_description TEXT
);
""")
# Part 2.3 Create a table for genes #
execute_sql_command("""
CREATE TABLE IF NOT EXISTS genes (
    gene_id INTEGER PRIMARY KEY AUTOINCREMENT,
    gene_name TEXT NOT NULL,
    gene_organism TEXT NOT NULL,
    gene_nucleotide_sequence TEXT,
    gene_description TEXT,
    enzyme_commission TEXT,
    pathway_name TEXT
);
""")    

In [13]:
# Part 3.1 Add & Print data from KEGG to the pathway table #
import sqlite3
import pandas as pd
execute_sql_command("DELETE FROM pathways;")
execute_sql_command("""
INSERT INTO pathways (pathway_name)
VALUES
('Glycolysis'),
('Citrate cycle (TCA cycle)'),
('Pentose phosphate pathway');
""")
conn = sqlite3.connect("lab3.db")
df = pd.read_sql("SELECT * FROM pathways;", conn)
print(df)

   pathway_id               pathway_name pathway_description
0          13                 Glycolysis                None
1          14  Citrate cycle (TCA cycle)                None
2          15  Pentose phosphate pathway                None


In [14]:
# Part 3.2 Add & Print data drom KEGG to the enzyme table #
execute_sql_command("""
INSERT INTO enzymes (enzyme_commission, enzyme_name)
VALUES
('2.7.1.1', 'hexokinase'),
('2.7.1.63', 'polyphosphate glucokinase'),
('5.3.1.9', 'glucose-6-phosphate isomerase'),
('4.2.1.13', 'fructose-bisphosphate aldolase'),
('1.1.1.42', 'isocitrate dehydrogenase'),
('2.3.3.1', 'citrate synthase'),
('4.2.1.3', 'aconitate hydratase'),
('4.1.1.32', 'phosphoenolpyruvate carboxykinase (GTP)'),
('1.1.1.44', '6-phosphogluconate dehydrogenase'),
('4.1.2.43', '3-hexulose-6-phosphate synthase'),
('5.3.1.6', 'ribose 5-phosphate isomerase A'),
('2.7.1.11', '6-phosphofructokinase');
""")
conn = sqlite3.connect("lab3.db")
df = pd.read_sql("SELECT * FROM enzymes;", conn)
print(df)

    enzyme_id enzyme_commission                              enzyme_name  \
0           1           2.7.1.1                               hexokinase   
1           2          2.7.1.63                polyphosphate glucokinase   
2           3           5.3.1.9            glucose-6-phosphate isomerase   
3           4          4.2.1.13           fructose-bisphosphate aldolase   
4           5          1.1.1.42                 isocitrate dehydrogenase   
5           6           2.3.3.1                         citrate synthase   
6           7           4.2.1.3                      aconitate hydratase   
7           8          4.1.1.32  phosphoenolpyruvate carboxykinase (GTP)   
8           9          1.1.1.44         6-phosphogluconate dehydrogenase   
9          10          4.1.2.43          3-hexulose-6-phosphate synthase   
10         11           5.3.1.6           ribose 5-phosphate isomerase A   
11         12          2.7.1.11                    6-phosphofructokinase   

   enzyme_d

In [ ]:
# Part 4 Populate gene database automatically #
# Part 4.1 Setup & Preparations #
from Bio import Entrez
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import sqlite3
Entrez.email = "s.xuan@berkeley.edu"
def save_enzyme(ec_number: str, organism: str, file: str) -> None:
    """
